In [1]:
import csv
import pickle
import urllib.request
from collections import Counter, defaultdict
from multiprocessing import Pool
from pathlib import Path

import ipadic
import numpy as np
from tqdm import tqdm

import vibrato

In [2]:
!ls ./../data/interim/

emo_count_10000.pickle	emo_dict_10000.pickle  emo_hash_10000.pickle
emo_count_10001.pickle	emo_dict_10001.pickle  emo_hash_10001.pickle
emo_count_1.0.pickle	emo_dict_1.0.pickle    emo_hash_1.0.pickle
emo_count_20000.pickle	emo_dict_20000.pickle  emo_hash_20000.pickle
emo_count_2.0.pickle	emo_dict_2.0.pickle    emo_hash_2.0.pickle
emo_count_30000.pickle	emo_dict_30000.pickle  emo_hash_30000.pickle
emo_count_end.pickle	emo_dict_end.pickle    emo_hash_end.pickle
emo_count.pickle	emo_dict.pickle        emo_hash.pickle


In [3]:
pwd

'/home/anosillus/git/github.com/anosillus/emotion_analysis/src'

In [4]:
!ls /home/anosillus/git/github.com/anosillus/emotion_analysis/data/interim/emo_dict_10000.pickle

/home/anosillus/git/github.com/anosillus/emotion_analysis/data/interim/emo_dict_10000.pickle


In [5]:
def read_data(name = '1000'):
    with open('./emo_dict_10000.pickle', "rb") as f:
        emo_dict = pickle.load(f)

    with open(f"./emo_hash_{name}.pickle", "rb") as f:
        hash_dict = pickle.load(f)

    with open(f"./emo_count_{name}.pickle", "rb") as f:
        count_dict = pickle.load(f)


    return (emo_dict, hash_dict, count_dict)

In [6]:
emo, hash, count = read_data()

In [7]:
melos =  '''
　メロスは激怒した。必ず、かの邪智暴虐じゃちぼうぎゃくの王を除かなければならぬと決意した。メロスには政治がわからぬ。メロスは、村の牧人である。笛を吹き、羊と遊んで暮して来た。けれども邪悪に対しては、人一倍に敏感であった。きょう未明メロスは村を出発し、野を越え山越え、十里はなれた此このシラクスの市にやって来た。メロスには父も、母も無い。女房も無い。十六の、内気な妹と二人暮しだ。この妹は、村の或る律気な一牧人を、近々、花婿はなむことして迎える事になっていた。結婚式も間近かなのである。メロスは、それゆえ、花嫁の衣裳やら祝宴の御馳走やらを買いに、はるばる市にやって来たのだ。先ず、その品々を買い集め、それから都の大路をぶらぶら歩いた。メロスには竹馬の友があった。セリヌンティウスである。今は此のシラクスの市で、石工をしている。その友を、これから訪ねてみるつもりなのだ。久しく逢わなかったのだから、訪ねて行くのが楽しみである。歩いているうちにメロスは、まちの様子を怪しく思った。ひっそりしている。もう既に日も落ちて、まちの暗いのは当りまえだが、けれども、なんだか、夜のせいばかりでは無く、市全体が、やけに寂しい。のんきなメロスも、だんだん不安になって来た。路で逢った若い衆をつかまえて、何かあったのか、二年まえに此の市に来たときは、夜でも皆が歌をうたって、まちは賑やかであった筈はずだが、と質問した。若い衆は、首を振って答えなかった。しばらく歩いて老爺ろうやに逢い、こんどはもっと、語勢を強くして質問した。老爺は答えなかった。メロスは両手で老爺のからだをゆすぶって質問を重ねた。老爺は、あたりをはばかる低声で、わずか答えた。
「王様は、人を殺します。」
「なぜ殺すのだ。」
「悪心を抱いている、というのですが、誰もそんな、悪心を持っては居りませぬ。」
「たくさんの人を殺したのか。」
「はい、はじめは王様の妹婿さまを。それから、御自身のお世嗣よつぎを。それから、妹さまを。それから、妹さまの御子さまを。それから、皇后さまを。それから、賢臣のアレキス様を。」
「おどろいた。国王は乱心か。」
「いいえ、乱心ではございませぬ。人を、信ずる事が出来ぬ、というのです。このごろは、臣下の心をも、お疑いになり、少しく派手な暮しをしている者には、人質ひとりずつ差し出すことを命じて居ります。御命令を拒めば十字架にかけられて、殺されます。きょうは、六人殺されました。」
　聞いて、メロスは激怒した。「呆あきれた王だ。生かして置けぬ。」
　メロスは、単純な男であった。買い物を、背負ったままで、のそのそ王城にはいって行った。たちまち彼は、巡邏じゅんらの警吏に捕縛された。調べられて、メロスの懐中からは短剣が出て来たので、騒ぎが大きくなってしまった。メロスは、王の前に引き出された。
「この短刀で何をするつもりであったか。言え！」暴君ディオニスは静かに、けれども威厳を以もって問いつめた。その王の顔は蒼白そうはくで、眉間みけんの皺しわは、刻み込まれたように深かった。
「市を暴君の手から救うのだ。」とメロスは悪びれずに答えた。
「おまえがか？」王は、憫笑びんしょうした。「仕方の無いやつじゃ。おまえには、わしの孤独がわからぬ。」
「言うな！」とメロスは、いきり立って反駁はんばくした。「人の心を疑うのは、最も恥ずべき悪徳だ。王は、民の忠誠をさえ疑って居られる。」
「疑うのが、正当の心構えなのだと、わしに教えてくれたのは、おまえたちだ。人の心は、あてにならない。人間は、もともと私慾のかたまりさ。信じては、ならぬ。」暴君は落着いて呟つぶやき、ほっと溜息ためいきをついた。「わしだって、平和を望んでいるのだが。」
「なんの為の平和だ。自分の地位を守る為か。」こんどはメロスが嘲笑した。「罪の無い人を殺して、何が平和だ。」
「だまれ、下賤げせんの者。」王は、さっと顔を挙げて報いた。「口では、どんな清らかな事でも言える。わしには、人の腹綿の奥底が見え透いてならぬ。おまえだって、いまに、磔はりつけになってから、泣いて詫わびたって聞かぬぞ。」
「ああ、王は悧巧りこうだ。自惚うぬぼれているがよい。私は、ちゃんと死ぬる覚悟で居るのに。命乞いなど決してしない。ただ、――」と言いかけて、メロスは足もとに視線を落し瞬時ためらい、「ただ、私に情をかけたいつもりなら、処刑までに三日間の日限を与えて下さい。たった一人の妹に、亭主を持たせてやりたいのです。三日のうちに、私は村で結婚式を挙げさせ、必ず、ここへ帰って来ます。」
「ばかな。」と暴君は、嗄しわがれた声で低く笑った。「とんでもない嘘うそを言うわい。逃がした小鳥が帰って来るというのか。」
「そうです。帰って来るのです。」メロスは必死で言い張った。「私は約束を守ります。私を、三日間だけ許して下さい。妹が、私の帰りを待っているのだ。そんなに私を信じられないならば、よろしい、この市にセリヌンティウスという石工がいます。私の無二の友人だ。あれを、人質としてここに置いて行こう。私が逃げてしまって、三日目の日暮まで、ここに帰って来なかったら、あの友人を絞め殺して下さい。たのむ、そうして下さい。」
　それを聞いて王は、残虐な気持で、そっと北叟笑ほくそえんだ。生意気なことを言うわい。どうせ帰って来ないにきまっている。この嘘つきに騙だまされた振りして、放してやるのも面白い。そうして身代りの男を、三日目に殺してやるのも気味がいい。人は、これだから信じられぬと、わしは悲しい顔して、その身代りの男を磔刑に処してやるのだ。世の中の、正直者とかいう奴輩やつばらにうんと見せつけてやりたいものさ。
「願いを、聞いた。その身代りを呼ぶがよい。三日目には日没までに帰って来い。おくれたら、その身代りを、きっと殺すぞ。ちょっとおくれて来るがいい。おまえの罪は、永遠にゆるしてやろうぞ。」
「なに、何をおっしゃる。」
「はは。いのちが大事だったら、おくれて来い。おまえの心は、わかっているぞ。」
　メロスは口惜しく、地団駄じだんだ踏んだ。ものも言いたくなくなった。
　竹馬の友、セリヌンティウスは、深夜、王城に召された。暴君ディオニスの面前で、佳よき友と佳き友は、二年ぶりで相逢うた。メロスは、友に一切の事情を語った。セリヌンティウスは無言で首肯うなずき、メロスをひしと抱きしめた。友と友の間は、それでよかった。セリヌンティウスは、縄打たれた。メロスは、すぐに出発した。初夏、満天の星である。
　メロスはその夜、一睡もせず十里の路を急ぎに急いで、村へ到着したのは、翌あくる日の午前、陽は既に高く昇って、村人たちは野に出て仕事をはじめていた。メロスの十六の妹も、きょうは兄の代りに羊群の番をしていた。よろめいて歩いて来る兄の、疲労困憊こんぱいの姿を見つけて驚いた。そうして、うるさく兄に質問を浴びせた。
「なんでも無い。」メロスは無理に笑おうと努めた。「市に用事を残して来た。またすぐ市に行かなければならぬ。あす、おまえの結婚式を挙げる。早いほうがよかろう。」
　妹は頬をあからめた。'''

In [8]:
import vibrato


def get_tokenizer():
    with open("./vibrato/ipadic-mecab-2_7_0/system.dic", "rb") as fp:
        dict_data = fp.read()

    return vibrato.Vibrato(dict_data)

In [9]:
tokenizer = get_tokenizer()

In [10]:
def read_stopwords() -> list[str]:

    if not Path("./../data/stopwords.pickle").exists():
        stopwords_url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
        sub_words_url = "https://gist.githubusercontent.com/ftnext/2e14fb57c96ca276ee4d28eccfcecd96/raw/f0b1557cd576fa223cb954fc983f5117b3372ac9/stop_words.txt"
        original_words = """ない ある える くる
あ あっ あまり あり ある あるいは あれ
い いい いう いく いずれ いっ いつ いる いわ
うち
え
お おい おけ および おら おり
か かけ かつ かつて かなり から が
き きっかけ
くる くん
こ こう ここ こと この これ ご ごと
さ さらに さん
し しか しかし しまう しまっ しよう
す すぐ すべて する ず
せ せい せる
そう そこ そして その それ それぞれ
た たい ただし たち ため たら たり だ だけ だっ
ち ちゃん
つ つい つけ つつ
て で でき できる です
と とき ところ とっ とも どう
な ない なお なかっ ながら なく なけれ なし なっ など なら なり なる
に にて
ぬ
ね
の のち のみ
は はじめ ば
ひと
ぶり
へ べき
ほか ほとんど ほど ほぼ
ま ます また まで まま
み
も もう もっ もと もの
や やっ
よ よう よく よっ より よる よれ
ら らしい られ られる
る
れ れる
を
ん
一
""".split()

        with urllib.request.urlopen(stopwords_url) as response:
            stopwords = [
                line.decode("utf-8").strip()
                for line in response
                if not line in ("", " ")
            ]
        with urllib.request.urlopen(sub_words_url) as response:
            sub_stopwords = [
                line.decode("utf-8").strip()
                for line in response
                if not line in ("", " ")
            ]
        stopwords = set(stopwords + sub_stopwords + original_words)

        with open("./../data/stopwords.pickle", "wb") as f:
            pickle.dump(stopwords, f)
    else:
        with open("./../data/stopwords.pickle", "rb") as f:
            stopwords = pickle.load(f)
    return stopwords

In [11]:
sw = read_stopwords()

In [12]:
MECAB_RESULT_INDEX = [
    "品詞",
    "品詞細分類1",
    "品詞細分類2",
    "品詞細分類3",
    "活用型",
    "活用形",
    "原形",
    "読み",
    "発音",
]

In [13]:
def wakachi(text="麩菓子は、麩を主材料とした日本の菓子。"):
    stopwords = read_stopwords()

    def is_target_words(word):
        return word not in stopwords and word.feature().split(",")[0] in (
            "動詞",
            "名詞",
            "形容詞",
            "連体詞",
        )

    tokenizer = get_tokenizer()

    # tokens = tokenizer.tokenize("社長は火星猫だ")
    # text = "社長は火星猫だ"
    # print(list(tokens))
    return [
        dict(zip(MECAB_RESULT_INDEX, word.feature().split(",")))
        for word in tokenizer.tokenize(text)
        if is_target_words(word)
    ]

In [14]:
a = wakachi(melos)

In [16]:
from fabric.colors  import red, green, yellow, blue, magenta, cyan, white

ModuleNotFoundError: No module named 'fabric.colors'

In [85]:
def print_emo(word, i):
    if i == 0:
        print(red(word))
    elif i == 1:
        print(green(word))
    elif i == 2:
        print(yello(word))
    elif i == 3:
        print(blue(word))
    elif i == 4:
        print(magenta(word))
    elif i == 5:
        print(cyan(word))

In [81]:
import math

In [84]:
for word in tokenizer.tokenize(melos):
    print(word)

    if word not in sw and word.feature().split(",")[0] in (
            "動詞",
            "名詞",
            "形容詞",
            "連体詞",
        ):
        #print(word.feature())
        a = emo.get(word.surface())
        try:
            if a:
                emotion_i = np.argmax(a)
        except Exception:
            emotion_i = np.argmax(a)
            



　
メロス
は
激怒
[0.         0.         0.35384615 0.         0.         0.
 0.         0.        ]
し
た
。
必ず
、
かの
邪智
暴虐
じ
ゃちぼうぎゃくの
王
[0.71345029 0.05555556 0.31578947 0.81111111 0.         0.
 0.05555556 0.        ]
を
除か
なけれ
ば
なら
ぬ
と
決意
し
た
。
メロス
に
は
政治
[0.         0.08630952 0.         0.         0.11904762 0.
 0.08333333 0.        ]
が
わから
ぬ
。
メロス
は
、
村
[0.20952381 0.33834586 0.36666667 0.25263158 0.         0.05263158
 0.15263158 0.        ]
の
牧人
で
ある
。
笛
を
吹き
、
羊
[0.88162393 0.         0.36410256 0.16666667 0.         0.
 0.         0.15555556]
と
遊ん
で
暮し
て
来
た
。
けれども
邪悪
に対して
は
、
人一倍
に
敏感
[0.         0.         0.0952381  0.         0.         0.
 0.04761905 0.        ]
で
あっ
た
。
きょう
[0.  0.5 0.  0.  0.  0.  0.5 0. ]
未明
[0.         0.         0.         0.03030303 0.         0.09090909
 0.         0.        ]
メロス
は
村
[0.20952381 0.33834586 0.36666667 0.25263158 0.         0.05263158
 0.15263158 0.        ]
を
出発
[0.14285714 0.5        0.34285714 0.21111111 0.         0.37777778
 0.11111111

In [48]:
emo

{'ぼけ': array([0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. ]),
 'する': array([150.57349104, 123.75394867, 178.37551612, 134.04680869,
         35.50960145,  73.02542369,  98.10553536,  24.40306455]),
 'てる': array([ 85.42580285,  73.34334202,  54.08441622, 139.97621913,
         16.53070464,  43.38282911,  47.49319005,  12.2880436 ]),
 'こんな': array([2.87250167, 3.37882868, 2.54594094, 5.9268265 , 1.38550704,
        2.35531222, 3.03742452, 0.12406015]),
 '時間': array([11.33919607, 16.70463853, 10.88448734, 17.71415979,  2.39841641,
         7.51346249,  7.56303227,  1.97103377]),
 '*': array([765.1773697 , 371.4160476 , 632.15206594, 627.45398879,
         74.57282296, 177.91386429, 234.23912832, 101.06068993]),
 'ある': array([26.99310806, 21.33081924, 31.51621271, 38.88734486,  5.00188471,
        12.54526391, 16.45490999,  5.84414962]),
 '食べる': array([20.55815426, 14.17773398, 39.70101533, 13.41505922,  1.02965169,
         5.8680457 ,  4.83758636,  2.70549436]),
 'でる': array([3.75518318, 6.959

In [ ]:
for i in wakachi(emo_data[0]):
    wakachi(emo_data[0])


In [ ]:
hash(frozenset(word_info.items())

In [40]:
hashee

{806849686802419705: 9,
 -6299155085364615957: 32,
 1679757765377551975: 11,
 6675980103434844221: 9,
 6618322385694622866: 98,
 -7122951311597656374: 5,
 1636385043774694757: 23,
 8050421592987045955: 43,
 -6731821776164073043: 3,
 702593289190013851: 81,
 -7275379056371060981: 2,
 2633096546254507108: 71,
 -8347263824763219639: 2100,
 -8050669740634084922: 54,
 5017115133786636526: 16,
 -7744004994457956655: 252,
 2953880185537920049: 88,
 1241638166586460169: 47,
 8751305337123266585: 5047,
 -8256506153760739172: 5,
 -3979289530853184414: 10,
 8192456558221534873: 2,
 -6834877296841803143: 5,
 -6266618363090038041: 5,
 4886217881293871372: 3,
 -7216240866610063644: 18,
 -6999364867226652777: 1,
 8904429167262540798: 2,
 5707649854027925987: 13,
 -3729129733858085447: 9,
 4615144799033046144: 13,
 9223251789070728589: 4,
 -3768951287158270707: 49,
 -4069722261645218564: 50,
 -2429476297755648148: 22,
 5700050303033038661: 20,
 8919615959050848624: 12,
 -2591791253929810799: 5,
 73655

In [39]:
emo

{806849686802419705: array([0.61731717, 0.05555556, 0.75309275, 0.27692308, 0.        ,
        0.        , 0.07692308, 0.03448276]),
 -6299155085364615957: array([2.71113516, 0.95490196, 3.96190476, 0.95650129, 0.42777778,
        0.38104575, 0.80287115, 1.03636364]),
 1679757765377551975: array([1.01666667, 0.78181818, 1.78571429, 0.325     , 0.        ,
        0.1       , 0.        , 0.        ]),
 6675980103434844221: array([1.58134921, 0.1       , 0.57222222, 1.24404762, 0.        ,
        0.        , 0.        , 0.07142857]),
 6618322385694622866: array([ 8.22392038, 10.46321871,  3.98625278,  6.20321155,  1.53071068,
         4.81856061,  9.65256118,  0.70192308]),
 -7122951311597656374: array([0.5       , 0.14285714, 0.        , 0.225     , 0.        ,
        0.3       , 0.125     , 0.        ]),
 1636385043774694757: array([1.46909611, 0.32125   , 0.88882784, 0.88558934, 0.24358974,
        0.71429627, 0.35096154, 0.02941176]),
 8050421592987045955: array([2.04050125, 1.630